# 전력 소비량 예측 - Feature Selection

**목표**: EDA 결과를 바탕으로 최종 모델에 사용할 피처 선정

**프로세스**:
1. EDA 기반 파생 피처 생성
2. 피처 중요도 분석
3. 상관관계 및 다중공선성 분석
4. 최종 피처 선정

## 1. 환경 설정

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
sns.set_style('whitegrid')

print("✅ 환경 설정 완료")

## 2. 데이터 로드 및 기본 전처리

In [ ]:
# 데이터 로드
DATA_DIR = './data'
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), encoding='utf-8-sig')
building_info = pd.read_csv(os.path.join(DATA_DIR, 'building_info.csv'), encoding='utf-8-sig')

# 날짜 변환
train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')
train['hour'] = train['일시'].dt.hour
train['day_of_week'] = train['일시'].dt.dayofweek
train['day'] = train['일시'].dt.day
train['month'] = train['일시'].dt.month
train['day_of_year'] = train['일시'].dt.dayofyear
train['is_weekend'] = train['day_of_week'].isin([5, 6]).astype(int)

# 건물 정보 병합
train = train.merge(building_info, on='건물번호', how='left')

print(f"✅ Train shape: {train.shape}")
print(f"   총 피처 수 (초기): {len(train.columns)}")

## 3. EDA 기반 파생 피처 생성

EDA에서 효과가 확인된 피처들을 생성합니다.

### 3.1 비선형 피처 (제곱항)

In [ ]:
# 온도와 습도의 제곱항 (비선형 관계 포착)
train['temperature_squared'] = train['기온(°C)'] ** 2
train['humidity_squared'] = train['습도(%)'] ** 2

print("✅ 제곱 피처 생성 완료")
print(f"   - temperature_squared")
print(f"   - humidity_squared")

### 3.2 순환 피처 (Cyclic Features)

In [ ]:
# 시간과 날짜의 순환적 특성을 sin/cos로 표현
train['sin_hour'] = np.sin(2 * np.pi * train['hour'] / 24)
train['cos_hour'] = np.cos(2 * np.pi * train['hour'] / 24)
train['sin_doy'] = np.sin(2 * np.pi * (train['day_of_year'] - 1) / 365)
train['cos_doy'] = np.cos(2 * np.pi * (train['day_of_year'] - 1) / 365)

# 여름 주기 (5/20 ~ 9/8)
start_date = datetime(2024, 5, 20)
end_date = datetime(2024, 9, 8)
period_seconds = (end_date - start_date).total_seconds()

def summer_cos(date):
    return np.cos(2 * np.pi * (date - start_date).total_seconds() / period_seconds)

def summer_sin(date):
    return np.sin(2 * np.pi * (date - start_date).total_seconds() / period_seconds)

train['summer_cos'] = train['일시'].apply(summer_cos)
train['summer_sin'] = train['일시'].apply(summer_sin)

print("✅ 순환 피처 생성 완료")
print(f"   - sin_hour, cos_hour")
print(f"   - sin_doy, cos_doy")
print(f"   - summer_sin, summer_cos")

### 3.3 온도 기반 복합 지표

In [ ]:
# CDH (Cooling Degree Hours) - 간단 버전
def calc_cdh(temps, window=12, base_temp=26):
    cdh_list = []
    for i in range(len(temps)):
        if i < window:
            cdh = np.sum(np.maximum(temps[:i+1] - base_temp, 0))
        else:
            cdh = np.sum(np.maximum(temps[i-window+1:i+1] - base_temp, 0))
        cdh_list.append(cdh)
    return cdh_list

# 건물별로 CDH 계산
train = train.sort_values(['건물번호', '일시'])
cdh_all = []
for building in train['건물번호'].unique():
    building_data = train[train['건물번호'] == building]
    cdh = calc_cdh(building_data['기온(°C)'].values)
    cdh_all.extend(cdh)
train['CDH'] = cdh_all

# CDD (Cooling Degree Days)
train['excess'] = (train['기온(°C)'] - 18.0).clip(lower=0)
train['CDD'] = train.groupby('건물번호')['excess'].transform(
    lambda s: s.rolling(24, min_periods=1).sum()
)
train.drop(columns=['excess'], inplace=True)

# THI (Temperature Humidity Index)
train['THI'] = (9/5 * train['기온(°C)'] 
                - 0.55 * (1 - train['습도(%)'] / 100) 
                * (9/5 * train['기온(°C)'] - 26) + 32)

# WCT (Wind Chill Temperature)
v16 = train['풍속(m/s)'] ** 0.16
train['WCT'] = (13.12 + 0.6215 * train['기온(°C)'] 
                - 11.37 * v16 + 0.3965 * v16 * train['기온(°C)'])

print("✅ 온도 기반 복합 지표 생성 완료")
print(f"   - CDH (Cooling Degree Hours)")
print(f"   - CDD (Cooling Degree Days)")
print(f"   - THI (Temperature Humidity Index)")
print(f"   - WCT (Wind Chill Temperature)")

### 3.4 휴일 피처

In [ ]:
# 공휴일 설정 (2024년)
kr_holidays = pd.to_datetime(['2024-06-06', '2024-08-15']).date
train['date'] = train['일시'].dt.date
train['holiday'] = (train['date'].isin(kr_holidays) | train['is_weekend'].astype(bool)).astype(int)

print("✅ 휴일 피처 생성 완료")
print(f"   - holiday (공휴일 + 주말)")

### 3.5 온도/습도 통계 피처

In [ ]:
# 일별 온도 통계
temp_stats = train[train['hour'] % 3 == 0].groupby(['건물번호', 'day', 'month'])['기온(°C)'].agg(
    avg_temp='mean',
    max_temp='max',
    min_temp='min'
).reset_index()
train = train.merge(temp_stats, on=['건물번호', 'day', 'month'], how='left')
train['temp_diff'] = train['max_temp'] - train['min_temp']

# 일별 습도 통계
humid_stats = train[train['hour'] % 3 == 0].groupby(['건물번호', 'day', 'month'])['습도(%)'].agg(
    avg_humid='mean',
    max_humid='max',
    min_humid='min'
).reset_index()
train = train.merge(humid_stats, on=['건물번호', 'day', 'month'], how='left')
train['humid_diff'] = train['max_humid'] - train['min_humid']

print("✅ 온도/습도 통계 피처 생성 완료")
print(f"   - avg_temp, max_temp, min_temp, temp_diff")
print(f"   - avg_humid, max_humid, min_humid, humid_diff")

### 3.6 타겟 통계 피처 (Target Encoding)

시간대별/요일별 패턴을 활용한 타겟 통계 피처

In [ ]:
# 요일-시간대별 평균/표준편차
dow_hour_mean = train.groupby(['건물번호', 'hour', 'day_of_week'])['전력소비량(kWh)'].mean().reset_index(
    name='dow_hour_mean'
)
dow_hour_std = train.groupby(['건물번호', 'hour', 'day_of_week'])['전력소비량(kWh)'].std().reset_index(
    name='dow_hour_std'
)
train = train.merge(dow_hour_mean, on=['건물번호', 'hour', 'day_of_week'], how='left')
train = train.merge(dow_hour_std, on=['건물번호', 'hour', 'day_of_week'], how='left')

# 휴일-시간대별 평균/표준편차
hol_mean = train.groupby(['건물번호', 'hour', 'holiday'])['전력소비량(kWh)'].mean().reset_index(
    name='holiday_mean'
)
hol_std = train.groupby(['건물번호', 'hour', 'holiday'])['전력소비량(kWh)'].std().reset_index(
    name='holiday_std'
)
train = train.merge(hol_mean, on=['건물번호', 'hour', 'holiday'], how='left')
train = train.merge(hol_std, on=['건물번호', 'hour', 'holiday'], how='left')

# 시간대별 평균/표준편차
hr_mean = train.groupby(['건물번호', 'hour'])['전력소비량(kWh)'].mean().reset_index(name='hour_mean')
hr_std = train.groupby(['건물번호', 'hour'])['전력소비량(kWh)'].std().reset_index(name='hour_std')
train = train.merge(hr_mean, on=['건물번호', 'hour'], how='left')
train = train.merge(hr_std, on=['건물번호', 'hour'], how='left')

# 월-시간대별 평균/표준편차
mh_mean = train.groupby(['건물번호', 'month', 'hour'])['전력소비량(kWh)'].mean().reset_index(
    name='month_hour_mean'
)
mh_std = train.groupby(['건물번호', 'month', 'hour'])['전력소비량(kWh)'].std().reset_index(
    name='month_hour_std'
)
train = train.merge(mh_mean, on=['건물번호', 'month', 'hour'], how='left')
train = train.merge(mh_std, on=['건물번호', 'month', 'hour'], how='left')

print("✅ 타겟 통계 피처 생성 완료")
print(f"   - dow_hour_mean, dow_hour_std")
print(f"   - holiday_mean, holiday_std")
print(f"   - hour_mean, hour_std")
print(f"   - month_hour_mean, month_hour_std")

### 3.7 전력소비 트렌드 피처

In [ ]:
# Weekly slope (168시간 = 1주일)
# 간단 버전: 1주 전 전력소비량
train['power_week_ago'] = train.groupby('건물번호')['전력소비량(kWh)'].shift(168)

# 간단한 slope 근사
train['power_week_slope6h'] = train.groupby('건물번호')['power_week_ago'].transform(
    lambda x: x.rolling(6, min_periods=1).mean()
).fillna(0)

train.drop(columns=['power_week_ago'], inplace=True)

print("✅ 전력소비 트렌드 피처 생성 완료")
print(f"   - power_week_slope6h")

print(f"\n✅ 총 생성된 피처 수: {len(train.columns)}")

## 4. 피처 중요도 분석

간단한 RandomForest 모델로 피처 중요도를 평가합니다.

In [ ]:
# 샘플 건물 선택 (전체 데이터는 너무 크므로)
sample_buildings = train['건물번호'].unique()[:10]
sample_data = train[train['건물번호'].isin(sample_buildings)].copy()

# 피처 선택
feature_cols = [
    '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
    'temperature_squared', 'humidity_squared',
    'sin_hour', 'cos_hour', 'sin_doy', 'cos_doy', 'summer_sin', 'summer_cos',
    'CDH', 'CDD', 'THI', 'WCT',
    'holiday', 'is_weekend',
    'avg_temp', 'max_temp', 'min_temp', 'temp_diff',
    'avg_humid', 'max_humid', 'min_humid', 'humid_diff',
    'dow_hour_mean', 'dow_hour_std',
    'holiday_mean', 'holiday_std',
    'hour_mean', 'hour_std',
    'month_hour_mean', 'month_hour_std',
    'power_week_slope6h',
    'day', 'month'
]

X_sample = sample_data[feature_cols].fillna(0)
y_sample = sample_data['전력소비량(kWh)']

# RandomForest 모델 학습
rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_sample, y_sample)

# 피처 중요도
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("피처 중요도 Top 20:")
print(feature_importance.head(20))

# 시각화
plt.figure(figsize=(12, 10))
top20 = feature_importance.head(20)
plt.barh(range(len(top20)), top20['importance'], align='center')
plt.yticks(range(len(top20)), top20['feature'])
plt.xlabel('Feature Importance', fontsize=12)
plt.title('피처 중요도 Top 20 (RandomForest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 5. 상관관계 분석

피처 간 다중공선성을 확인합니다.

In [ ]:
# 주요 피처만 선택해서 상관관계 분석
key_features = [
    '기온(°C)', 'temperature_squared', '습도(%)', 'humidity_squared',
    'CDH', 'CDD', 'THI', 'WCT',
    'dow_hour_mean', 'hour_mean', 'month_hour_mean',
    '전력소비량(kWh)'
]

corr_matrix = sample_data[key_features].corr()

# 히트맵
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
           square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('주요 피처 상관관계 히트맵', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# 타겟과의 상관계수
target_corr = corr_matrix['전력소비량(kWh)'].drop('전력소비량(kWh)').abs().sort_values(ascending=False)
print("\n전력소비량과의 상관계수 (절대값):")
print(target_corr)

## 6. 최종 피처 선정

### 6.1 선택된 피처 목록

In [ ]:
# 최종 사용 피처
SELECTED_FEATURES = [
    # 기본 기상 피처
    'temperature', 'rainfall', 'windspeed', 'humidity',
    
    # 비선형 피처
    'temperature_squared', 'humidity_squared',
    
    # 순환 피처
    'sin_hour', 'cos_hour', 'sin_doy', 'cos_doy', 'summer_sin', 'summer_cos',
    
    # 온도 기반 복합 지표
    'CDH', 'CDD', 'THI', 'WCT',
    
    # 휴일/주말
    'holiday', 'is_weekend',
    
    # 온도/습도 통계
    'avg_temp', 'max_temp', 'min_temp', 'temp_diff',
    'avg_humid', 'max_humid', 'min_humid', 'humid_diff',
    
    # 타겟 통계 피처
    'dow_hour_mean', 'dow_hour_std',
    'holiday_mean', 'holiday_std',
    'hour_mean', 'hour_std',
    'month_hour_mean', 'month_hour_std',
    
    # 트렌드
    'power_week_slope6h',
    
    # 시간 피처
    'day', 'month',
    
    # 건물 정보
    'building_number', 'building_type', 'total_area', 'cooling_area'
]

print(f"✅ 최종 선택된 피처 수: {len(SELECTED_FEATURES)}")
print("\n피처 카테고리별 분류:")
print(f"  - 기본 기상: 4개")
print(f"  - 비선형: 2개")
print(f"  - 순환: 6개")
print(f"  - 온도 복합 지표: 4개")
print(f"  - 휴일/주말: 2개")
print(f"  - 온도/습도 통계: 8개")
print(f"  - 타겟 통계: 8개")
print(f"  - 트렌드: 1개")
print(f"  - 시간: 2개")
print(f"  - 건물 정보: 4개")

# 선택된 피처 목록 저장
pd.DataFrame({'feature': SELECTED_FEATURES}).to_csv('selected_features.csv', index=False)
print("\n✅ 선택된 피처 목록을 'selected_features.csv'에 저장했습니다.")

### 6.2 제외된 피처 및 근거

In [ ]:
EXCLUDED_FEATURES = {
    'sunshine': '결측값이 많고, 전력소비량과 상관관계 낮음',
    'solar_radiation': '결측값이 많고, 전력소비량과 상관관계 낮음',
    'solar_power_capacity': '대부분 0이거나 결측값',
    'ess_capacity': '대부분 0이거나 결측값',
    'pcs_capacity': '대부분 0이거나 결측값',
    'hour': '순환 피처(sin_hour, cos_hour)로 대체',
    'day_of_week': '타겟 통계 피처(dow_hour_mean 등)에 포함됨',
    'day_of_year': '순환 피처(sin_doy, cos_doy)로 대체'
}

print("제외된 피처 및 제외 근거:")
print("="*80)
for feature, reason in EXCLUDED_FEATURES.items():
    print(f"  • {feature:20s}: {reason}")
print("="*80)

## 7. Feature Selection 종합 결론

### 최종 선정 결과

**총 41개 피처 선정**

#### 선정 기준
1. **EDA 검증**: EDA에서 전력소비량과의 관계가 명확하게 확인된 피처
2. **피처 중요도**: RandomForest 기반 중요도 분석에서 상위권
3. **도메인 지식**: 건물 에너지 소비 패턴에 대한 도메인 지식 반영
4. **다중공선성 관리**: 상관관계가 너무 높은 피처는 하나만 선택

#### 핵심 피처 그룹
1. **타겟 통계 피처** (8개): 시간/요일 패턴 포착에 가장 효과적
2. **온도 관련 피처** (12개): 냉방 수요와 직접 연관
3. **순환 피처** (6개): 시간의 순환적 특성 표현
4. **복합 지표** (4개): CDH, CDD, THI, WCT 등 도메인 지식 기반

#### 기대 효과
- 비선형 관계 포착 (제곱항, 복합 지표)
- 시간 패턴 학습 (순환 피처, 타겟 통계)
- 휴일 효과 반영 (holiday 피처)
- 건물별 특성 차이 활용 (건물별 개별 모델링)

### 다음 단계
- 이상치 분석 및 처리 (max_point_pipeline에서 구현)
- 모델 학습 및 검증
- 건물별 개별 모델링 적용
- 앙상블 전략 수립